# Capstone Project - The Battle of Neighborhoods
### 1. Introduction
#### 1.1 Background

#####  We will cluster New York City and the city of Toronto neighbourghoods. In the clusters we will see how similar or dissimilar they are.  For this task we select  New York City borough Queens.  Based on wikipedia information „Queens is the most ethnically diverse urban area in the world It is the most ethnically diverse county in the United States.“.  In Toronto  we will choose borough North York.  Based on wikipedia information „North York is highly multicultural and diverse. In 2016, 56% of North York's residents were not born in Canada, and 60% were classified as belonging to a visible minority.The neighbourhoods of North York are highly diverse, inhabited by people of many different cultures.“
#### 1.2 Problem
##### We have to gather publically available data and merge it with the Foursquare API to explore neighborhoods in New York City and Toronto. We will use the explore function to get the most common venue categories in each neighborhood, and then use this feature to group the neighborhoods into clusters. We will use the k-means clustering algorithm to complete this task.  From Foursquare API  explore endpoint we choose section „food“, because in Canada Foursquare API  return more less data in compare with USA.    
#### 1.3 Interest
##### Similarities or dissimilarities between cities and they neighborhoods could be very useful for companies whose trying to expand they markets. Lets imagine that company has food business in New York borough Queens and wish to explore possibilities in Toronto borough North York.   Based on our report will be possibble to choose neigbourghoods where the same food type industries exist  or not.  


In [42]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from urllib.request import urlopen
from bs4 import BeautifulSoup
import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
#import folium # map rendering library
%matplotlib inline

### 2. Data section
#### 2.1 New York data

#####  We receive New York neighbourghoods data from   https://cocl.us/new_york_dataset.    Data exist on JSON format so we have to parse data and put into our dataframe. Finally data format will be 

##### Borough	Neighborhood	Latitude	Longitude

#### 2.2 Toronto data 

#####  We receive Toronto neighbourghoods data from web page https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M.  To gather data we will us    Beautiful Soup  a Python library for pulling data out of HTML .   Because in this page we don't have information about Lattitude and Longitude we download csv file  http://cocl.us/Geospatial_data/Geospatial_Coordinates.csv  and merge this  data to format the same we  used in New York :

##### Borough	Neighborhood	Latitude	Longitude

#### 2.3 Toronto North York  and New York Queens data 
##### Finally we merge Toronto and NewYork  dataframes leaving neigborhood  whoose depends to Toronto North York  and New York Queens  boroughs.  Data format the same:

##### Borough	Neighborhood	Latitude	Longitude   

#### 2.4 Foursquare data 

#####  Use the Foursquare API to explore neighborhoods in New York City and Toronto. We  will use the explore function to get the most common venue categories in each neighborhood .  We add additional parameter in Foursquare Explore endpoint.   Section parameter "food".  So we on request receive information based on food industry.  Finnally managed data  will be formated 

#####  Neighborhood	1st Most Common Venue	2nd Most Common Venue	3rd Most Common Venue ....

#####  With this data  we have to do clustering tasks  

In [43]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


In [44]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)
neighborhoods_data = newyork_data['features']

In [45]:
!wget -O Geospatial_Coordinates.csv http://cocl.us/Geospatial_data/Geospatial_Coordinates.csv

--2019-06-16 10:45:29--  http://cocl.us/Geospatial_data/Geospatial_Coordinates.csv
Resolving cocl.us (cocl.us)... 159.8.72.228
Connecting to cocl.us (cocl.us)|159.8.72.228|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cocl.us/Geospatial_data/Geospatial_Coordinates.csv [following]
--2019-06-16 10:45:29--  https://cocl.us/Geospatial_data/Geospatial_Coordinates.csv
Connecting to cocl.us (cocl.us)|159.8.72.228|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-06-16 10:45:30--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 185.235.236.197
Connecting to ibm.box.com (ibm.box.com)|185.235.236.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--

In [46]:
# Read Toronto cordinates
df_csv = pd.read_csv('Geospatial_Coordinates.csv')

In [47]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html = urlopen(url)
soup = BeautifulSoup(html, 'lxml')
type(soup)

bs4.BeautifulSoup

In [48]:
# Move all Toronto table wiki to data 
data = []
my_string = ''
table = soup.find('table', attrs={'class':'wikitable sortable'})
table_body = table.find('tbody')

rows = table_body.find_all('tr')
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    #my_string = ''.join(cols)
    #my_list = my_string.split(",")
    data.append([ele for ele in cols if ele]) # Get rid of empty values

# normalyse data
for data_index in range(len(data)):
    if len(data[data_index]) == 3:
        postcode = data[data_index][0]
        borough = data[data_index][1]
        if borough != 'Not assigned':
            neighbourghood = data[data_index][2]
            if neighbourghood == 'Not assigned':
                neighbourghood = borough
        if data_index>0:
            for partial_index in range(data_index):
                if len(data[partial_index]) == 3:
                    if ((data[partial_index][0] == postcode) and (data[partial_index][1] == borough)):
                        data[partial_index][2] = data[partial_index][2] + ', ' + neighbourghood
                        data[data_index][1] = 'Not assigned' # Mark as 'Not assigned' for future remove

# Create data frame  aliasing as pd and move from data negbourghoods to df

df = pd.DataFrame()
df = pd.DataFrame(columns=['Postal Code','Borough', 'Neighbourghood'])

for data_index in range(len(data)):
    if len(data[data_index]) == 3:
        postcode = data[data_index][0]
        borough = data[data_index][1]
        if borough != 'Not assigned':
            neighbourghood = data[data_index][2]
            if neighbourghood == 'Not assigned':
                neighbourghood = borough
            #  Append a new value to dataframe, remove all values where Foursquare dont have info
            if borough == 'North York':
                if (neighbourghood != 'Bayswater') \
                    and (neighbourghood != 'Bayview Village') \
                    and neighbourghood != 'Brookville' \
                    and neighbourghood != 'CFB Toronto, Downsview East' \
                    and neighbourghood != 'Downsview Central' \
                    and neighbourghood != 'Silver Hills, York Mills' \
                    and neighbourghood != 'Somerville' \
                    and neighbourghood != 'York Mills West' \
                    and neighbourghood != 'Downsview Northwest' \
                    and neighbourghood != 'Emery, Humberlea' \
                    and neighbourghood != 'Victoria Village' \
                    and neighbourghood != 'Newtonbrook, Willowdale': 
                    df = df.append({'Postal Code' : postcode , 'Borough' : borough, 'Neighbourghood' : neighbourghood} , ignore_index=True)


In [49]:
# Add coordinates to Toronto Neighbourghood
neighborhoods = pd.merge(df, df_csv, on='Postal Code',how='left')


Loop through the data and fill the dataframe one row at a time to append New York data to Toronto.

In [50]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    #  Append a new value to dataframe, remove all values where Foursquare dont have info 
    if borough =='Queens':
        if (neighborhood_name != 'Bayswater') \
            and (neighborhood_name != 'Bayview Village') \
            and neighborhood_name != 'Brookville' \
            and neighborhood_name != 'CFB Toronto, Downsview East' \
            and neighborhood_name != 'Downsview Central' \
            and neighborhood_name != 'Silver Hills, York Mills' \
            and neighborhood_name != 'Somerville' \
            and neighborhood_name != 'York Mills West' \
            and neighborhood_name != 'Downsview Northwest' \
            and neighborhood_name != 'Emery, Humberlea' \
            and neighborhood_name != 'Neponsit' \
            and neighborhood_name != 'Newtonbrook, Willowdale':         
            neighborhoods = neighborhoods.append({'Postal Code' : 'NYC',
                                                  'Borough': borough,
                                                  'Neighbourghood': neighborhood_name,
                                                  'Latitude': neighborhood_lat,
                                                  'Longitude': neighborhood_lon}, ignore_index=True)
neighborhoods

,Postal Code,Borough,Neighbourghood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
2,M3B,North York,Don Mills North,43.745906,-79.352188
3,M6B,North York,Glencairn,43.709577,-79.445073
4,M3C,North York,"Flemingdon Park, Don Mills South",43.725900,-79.340923
5,M2H,North York,Hillcrest Village,43.803762,-79.363452
6,M3H,North York,"Bathurst Manor, Downsview North, Wilson Heights",43.754328,-79.442259
7,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556
8,M3J,North York,"Northwood Park, York University",43.767980,-79.487262
9,M3L,North York,Downsview West,43.739015,-79.506944


In [51]:
# Define Four square credentials
CLIENT_ID = 'X5SGFPAZDMORJTH5U0KC4HQ5JXEM2WJOSNVVVZ2WUBJI2YCH' # your Foursquare ID
CLIENT_SECRET = 'DLAOFVCFTRY0X5ID0HUMWT0OUPMHXR1S4T2MMJ04JN5HMYGL' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100
section = 'food'

In [52]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [53]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&section={}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng,
            section,
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [54]:
# New York and Toronto venues

nyc_tor_venues = getNearbyVenues(names=neighborhoods['Neighbourghood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )

Parkwoods
Lawrence Heights, Lawrence Manor
Don Mills North
Glencairn
Flemingdon Park, Don Mills South
Hillcrest Village
Bathurst Manor, Downsview North, Wilson Heights
Fairview, Henry Farm, Oriole
Northwood Park, York University
Downsview West
Downsview, North Park, Upwood Park
Humber Summit
Bedford Park, Lawrence Manor East
Willowdale South
Willowdale West
Astoria
Woodside
Jackson Heights
Elmhurst
Howard Beach
Corona
Forest Hills
Kew Gardens
Richmond Hill
Flushing
Long Island City
Sunnyside
East Elmhurst
Maspeth
Ridgewood
Glendale
Rego Park
Woodhaven
Ozone Park
South Ozone Park
College Point
Whitestone
Bayside
Auburndale
Little Neck
Douglaston
Glen Oaks
Bellerose
Kew Gardens Hills
Fresh Meadows
Briarwood
Jamaica Center
Oakland Gardens
Queens Village
Hollis
South Jamaica
St. Albans
Rochdale
Springfield Gardens
Cambria Heights
Rosedale
Far Rockaway
Broad Channel
Breezy Point
Steinway
Beechhurst
Bay Terrace
Edgemere
Arverne
Rockaway Beach
Murray Hill
Floral Park
Holliswood
Jamaica Estate

In [55]:
# Size of resulting database
print(nyc_tor_venues.shape)
print('There are {} uniques categories.'.format(len(nyc_tor_venues['Venue Category'].unique())))
nyc_tor_venues.head()


(1754, 7)
There are 95 uniques categories.


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,KFC,43.754387,-79.333021,Fast Food Restaurant
1,Parkwoods,43.753259,-79.329656,Bella Vita Catering & Private Chef Service,43.756651,-79.331524,BBQ Joint
2,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763,Lac Vien Vietnamese Restaurant,43.721259,-79.468472,Vietnamese Restaurant
3,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763,Hot Dog Cart,43.718331,-79.465870,Hot Dog Joint
4,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763,Hot dog Stand @Stitches warehouse,43.718646,-79.464019,Hot Dog Joint


In [56]:
# Check how many venues were returned for each neighborhood
nyc_tor_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Arverne,5,5,5,5,5,5
Astoria,84,84,84,84,84,84
Astoria Heights,11,11,11,11,11,11
Auburndale,12,12,12,12,12,12
"Bathurst Manor, Downsview North, Wilson Heights",9,9,9,9,9,9
Bay Terrace,13,13,13,13,13,13
Bayside,56,56,56,56,56,56
"Bedford Park, Lawrence Manor East",21,21,21,21,21,21
Beechhurst,6,6,6,6,6,6


In [57]:
# one hot encoding
nyc_tor_onehot = pd.get_dummies(nyc_tor_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
nyc_tor_onehot['Neighborhood'] = nyc_tor_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [nyc_tor_onehot.columns[-1]] + list(nyc_tor_onehot.columns[:-1])
nyc_tor_onehot = nyc_tor_onehot[fixed_columns]

nyc_tor_onehot.head()

,Neighborhood,Afghan Restaurant,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bistro,...,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Thai Restaurant,Tibetan Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wings Joint
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Lawrence Heights, Lawrence Manor",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,"Lawrence Heights, Lawrence Manor",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Lawrence Heights, Lawrence Manor",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [58]:
nyc_tor_grouped = nyc_tor_onehot.groupby('Neighborhood').mean().reset_index()
nyc_tor_grouped

,Neighborhood,Afghan Restaurant,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bistro,...,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Thai Restaurant,Tibetan Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wings Joint
0,Arverne,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Astoria,0.000000,0.000000,0.000000,0.000000,0.000000,0.011905,0.023810,0.083333,0.000000,...,0.0,0.000000,0.000000,0.000000,0.011905,0.000000,0.000000,0.011905,0.000000,0.000000
2,Astoria Heights,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.090909,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Auburndale,0.000000,0.083333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,"Bathurst Manor, Downsview North, Wilson Heights",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,Bay Terrace,0.000000,0.153846,0.000000,0.000000,0.076923,0.000000,0.000000,0.076923,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Bayside,0.000000,0.071429,0.000000,0.000000,0.017857,0.000000,0.017857,0.017857,0.017857,...,0.0,0.000000,0.000000,0.000000,0.017857,0.000000,0.000000,0.000000,0.017857,0.000000
7,"Bedford Park, Lawrence Manor East",0.000000,0.047619,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.095238,0.000000,0.000000,0.000000,0.000000,0.000000
8,Beechhurst,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Bellaire,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [59]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [60]:
# Top 5 venues for each neighbourghood
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = nyc_tor_grouped['Neighborhood']

for ind in np.arange(nyc_tor_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(nyc_tor_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Arverne,Sandwich Place,Pizza Place,Thai Restaurant,Donut Shop,Falafel Restaurant
1,Astoria,Middle Eastern Restaurant,Bakery,Deli / Bodega,Greek Restaurant,Indian Restaurant
2,Astoria Heights,Deli / Bodega,Chinese Restaurant,Italian Restaurant,Burger Joint,Bakery
3,Auburndale,Korean Restaurant,Deli / Bodega,Italian Restaurant,American Restaurant,Diner
4,"Bathurst Manor, Downsview North, Wilson Heights",Pizza Place,Deli / Bodega,Fried Chicken Joint,Chinese Restaurant,Restaurant
